### 📘 Lesson 6: CFPP Flexibility

<div style="display: flex; align-items: center; justify-content: space-between;">
  <div>
    <h3>Course presenters</h3>
    <ul>
      <li><strong>Priyesh Gosai</strong> - Energy Systems Modeler and Training Coordinator</li>
      <li><strong>Dr. Fabian Hofmann</strong> - Senior Optimization and Energy System Modelling Expert</li>
    </ul>
  </div>
  <div>
    <a href="https://openenergytransition.org/index.html">
      <img src="https://openenergytransition.org/assets/img/oet-logo-red-n-subtitle.png" height="60" alt="OET">
    </a>
  </div>
</div>


##### 🎯 Learning Objectives  



* Introduce participants to the PyPSA toolbox.  
* Provide details of relevant components.  
* Build and solve a simple PyPSA model.  
* Review the data structures for static and time-series data.  
* Analyze the results.  

The content also includes references to other toolboxes such as `numpy`, `pandas`, `matplotlib`, and `plotly`, but only covers functions relevant to a PyPSA workflow.  

📌 Participants unfamiliar with these toolboxes are encouraged to explore online videos or courses for deeper learning. 🎥📚  

---

Modelling reserves


- Lesson 5 - SA Network - Hourly
- Lesson 6 - SA Network - 4 second (battery and gas engine)
- Lesson 7 - Hydro
- Lesson 8 - Market modelling (EU)




In [ ]:
# Google Colab users
# Remove the comments in the rows below to set up your notebook.

# from google.colab import drive
# import os

# drive.mount('/content/drive')
# os.chdir('/content/drive/MyDrive/psfo_2025/mec4131z/')

In [ ]:
# from colab_scripts import install_colab_dependencies
# install_colab_dependencies()

In [ ]:
import pypsa
import pandas as pd
import numpy as np
from training_scripts import *


In [ ]:
input_file_name = 'data/Lesson4_solution.xlsx'
path = convert_selected_sheets_to_csv(input_file_name, 'lesson4_csv_folder')
network = pypsa.Network(path)

INFO:root:Converted snapshots to CSV.
INFO:root:Converted generators-p_max_pu to CSV.
INFO:root:Converted loads-p_set to CSV.
INFO:root:Converted storage_units-inflow to CSV.
INFO:root:Conversion complete. CSV files are saved in 'lesson4_csv_folder'
INFO:root:Excel file closed successfully.


### 
---